In [ ]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-20 03:25:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.59MB/s    in 0.2s    

2020-08-20 03:25:56 (4.59 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://databootcamp-uci.s3-us-west-1.amazonaws.com/H-query.json"
spark.sparkContext.addFile(url)
df = spark.read.json(SparkFiles.get("H-query.json"))
df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attributes: struct (nullable = true)
 |    |    |    |-- ALAND: long (nullable = true)
 |    |    |    |-- AWATER: long (nullable = true)
 |    |    |    |-- BLKGRPCE: string (nullable = true)
 |    |    |    |-- COUNTYFP: string (nullable = true)
 |    |    |    |-- FUNCSTAT: string (nullable = true)
 |    |    |    |-- GEOID: string (nullable = true)
 |    |    |    |-- GEOID_Data: string (nullable = true)
 |    |    |    |-- H07B25003e2: long (nullable = true)
 |    |    |    |-- H07B25003e3: long (nullable = true)
 |    |    |    |-- H18B25081e1: long (nullable = true)
 |    |    |    |-- H18B25081e2: long (nullable = true)
 |    |    |    |-- H18B25081e4: long (nullable = true)
 |    |    |    |-- H18B25081e5: long (nullable = true)
 |    |    |    |-- H18B25081e6: long (nullable = true)
 |    |    |    |-- H18B25081e7: long (nullable = true)
 |    |    |    |-- H18B25081e8: 

In [ ]:
df.dtypes

[('features',
  'array<struct<attributes:struct<ALAND:bigint,AWATER:bigint,BLKGRPCE:string,COUNTYFP:string,FUNCSTAT:string,GEOID:string,GEOID_Data:string,H07B25003e2:bigint,H07B25003e3:bigint,H18B25081e1:bigint,H18B25081e2:bigint,H18B25081e4:bigint,H18B25081e5:bigint,H18B25081e6:bigint,H18B25081e7:bigint,H18B25081e8:bigint,H18B25082e1:bigint,H18B25082e2:bigint,H18B25082e3:bigint,INTPTLAT:string,INTPTLON:string,MTFCC:string,NAMELSAD:string,OBJECTID:bigint,STATEFP:string,Shape__Area:double,Shape__Length:double,TRACTCE:string>,geometry:struct<rings:array<array<array<double>>>>>>'),
 ('fields',
  'array<struct<alias:string,defaultValue:string,domain:string,length:bigint,name:string,sqlType:string,type:string>>'),
 ('geometryProperties',
  'struct<shapeAreaFieldName:string,shapeLengthFieldName:string,units:string>'),
 ('geometryType', 'string'),
 ('globalIdFieldName', 'string'),
 ('objectIdFieldName', 'string'),
 ('spatialReference', 'struct<latestWkid:bigint,wkid:bigint>'),
 ('uniqueIdFiel

In [ ]:
df1 = df.select("features").collect()

In [ ]:
df1

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df2 = df.select("fields").collect()
df2[0][0]

[Row(alias='FID', defaultValue=None, domain=None, length=None, name='OBJECTID', sqlType='sqlTypeOther', type='esriFieldTypeOID'),
 Row(alias='STATEFP', defaultValue=None, domain=None, length=2, name='STATEFP', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='COUNTYFP', defaultValue=None, domain=None, length=3, name='COUNTYFP', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='TRACTCE', defaultValue=None, domain=None, length=6, name='TRACTCE', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='BLKGRPCE', defaultValue=None, domain=None, length=1, name='BLKGRPCE', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='GEOID', defaultValue=None, domain=None, length=12, name='GEOID', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='NAMELSAD', defaultValue=None, domain=None, length=13, name='NAMELSAD', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='MTFCC', defaultValue=None, domain=None, length=5, name='

In [ ]:
df3 = df.select("geometryProperties").collect()
df3

[Row(geometryProperties=Row(shapeAreaFieldName='Shape__Area', shapeLengthFieldName='Shape__Length', units='esriMeters'))]

In [ ]:
df4 = df.select("geometryType").collect()
df4

[Row(geometryType='esriGeometryPolygon')]

In [ ]:
df5 = df.select("globalIdFieldName").collect()
df5

[Row(globalIdFieldName='')]

In [ ]:
df6 = df.select("objectIdFieldName").collect()
df6

[Row(objectIdFieldName='OBJECTID')]

In [ ]:
df7 = df.select("spatialReference").collect()
df7

[Row(spatialReference=Row(latestWkid=4326, wkid=4326))]

In [ ]:
df8 = df.select("uniqueIdField").collect()
df8

[Row(uniqueIdField=Row(isSystemMaintained=True, name='OBJECTID'))]

In [ ]:
temp_df = df1[0][0]
temp_df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
temp_df[0][0].TRACTCE

'086902'

In [ ]:
clean_data = []

In [ ]:
our_array = ["06059063907", "06059063805", "06059063806", "06059063903", "06059063908", "06059063807", "06059063701", "06059063808", "06059063905", "06059063906", "06059063102", "06059063702", "06059063604", "06059063605", "06059063802", "06059063201", "06059063202", "06059063902", "06059063301", "06059062610", "06059063601", "06059063103", "06059063904", "06059063803", "06059063302" ]

In [ ]:
for row in df1[0][0]:
  a = str(row[0].GEOID)
  if a[:-1] in our_array:
    clean_data.append(row[0])

In [ ]:
clean_data

[Row(ALAND=337300, AWATER=0, BLKGRPCE='3', COUNTYFP='059', FUNCSTAT='S', GEOID='060590632013', GEOID_Data='15000US060590632013', H07B25003e2=141, H07B25003e3=547, H18B25081e1=141, H18B25081e2=110, H18B25081e4=0, H18B25081e5=23, H18B25081e6=0, H18B25081e7=87, H18B25081e8=31, H18B25082e1=128300000, H18B25082e2=97100000, H18B25082e3=31200000, INTPTLAT='+33.6480393', INTPTLON='-117.9063376', MTFCC='G5030', NAMELSAD='Block Group 3', OBJECTID=405, STATEFP='06', Shape__Area=488002.08203125, Shape__Length=2966.66302086031, TRACTCE='063201'),
 Row(ALAND=686935, AWATER=0, BLKGRPCE='1', COUNTYFP='059', FUNCSTAT='S', GEOID='060590638071', GEOID_Data='15000US060590638071', H07B25003e2=301, H07B25003e3=979, H18B25081e1=301, H18B25081e2=202, H18B25081e4=41, H18B25081e5=63, H18B25081e6=0, H18B25081e7=98, H18B25081e8=99, H18B25082e1=260495000, H18B25082e2=187350000, H18B25082e3=73145000, INTPTLAT='+33.6707969', INTPTLON='-117.9274147', MTFCC='G5030', NAMELSAD='Block Group 1', OBJECTID=428, STATEFP='06'

In [ ]:
new_df = spark.createDataFrame(clean_data)

In [ ]:
new_df.show()

+------+------+--------+--------+--------+------------+-------------------+-----------+-----------+-----------+-----------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+-----------+-----------+----------

In [ ]:
panda_df = new_df.toPandas()

In [ ]:
panda_df

,ALAND,AWATER,BLKGRPCE,COUNTYFP,FUNCSTAT,GEOID,GEOID_Data,H07B25003e2,H07B25003e3,H18B25081e1,H18B25081e2,H18B25081e4,H18B25081e5,H18B25081e6,H18B25081e7,H18B25081e8,H18B25082e1,H18B25082e2,H18B25082e3,INTPTLAT,INTPTLON,MTFCC,NAMELSAD,OBJECTID,STATEFP,Shape__Area,Shape__Length,TRACTCE
0,337300,0,3,059,S,060590632013,15000US060590632013,141,547,141,110,0,23,0,87,31,128300000.0,97100000.0,31200000.0,+33.6480393,-117.9063376,G5030,Block Group 3,405,06,4.880021e+05,2966.663021,063201
1,686935,0,1,059,S,060590638071,15000US060590638071,301,979,301,202,41,63,0,98,99,260495000.0,187350000.0,73145000.0,+33.6707969,-117.9274147,G5030,Block Group 1,428,06,9.943708e+05,4988.515566,063807
2,209303,0,2,059,S,060590637022,15000US060590637022,8,313,8,0,0,0,0,0,8,NaN,NaN,NaN,+33.6409341,-117.9214514,G5030,Block Group 2,515,06,3.027684e+05,2518.881817,063702
3,323900,0,2,059,S,060590633022,15000US060590633022,86,331,86,49,9,9,0,31,37,82280000.0,48500000.0,33780000.0,+33.6321488,-117.9106383,G5030,Block Group 2,641,06,4.684434e+05,2903.018625,063302
4,323906,0,1,059,S,060590633021,15000US060590633021,300,70,300,137,0,14,0,123,163,415600000.0,169550000.0,246050000.0,+33.6349404,-117.9078651,G5030,Block Group 1,754,06,4.684838e+05,2903.205471,063302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,625083,0,3,059,S,060590639063,15000US060590639063,296,635,296,221,32,18,0,171,75,130352000.0,96657000.0,33695000.0,+33.6601360,-117.9041334,G5030,Block Group 3,1734,06,9.046137e+05,4487.184195,063906
72,835355,0,3,059,S,060590639023,15000US060590639023,0,474,0,0,0,0,0,0,0,NaN,NaN,NaN,+33.6713146,-117.9129663,G5030,Block Group 3,1804,06,1.209237e+06,4531.540365,063902
73,426535,0,4,059,S,060590639024,15000US060590639024,20,612,20,20,0,20,0,0,0,NaN,NaN,NaN,+33.6781853,-117.9129197,G5030,Block Group 4,1805,06,6.175345e+05,3772.653635,063902
74,548656,0,3,059,S,060590639053,15000US060590639053,414,199,414,327,10,53,0,264,87,298903000.0,226478000.0,72425000.0,+33.6600313,-117.9114501,G5030,Block Group 3,1807,06,7.940139e+05,3692.445287,063905


In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [ ]:
panda_df.to_csv('cleaned_H-query.csv')

In [ ]:
!cp cleaned_H-query.csv "drive/My Drive/Final Project/"

cp: cannot stat 'cleaned_H-query.csv': No such file or directory
